
# Sauk-Suiattle Sediment Model

This data is compiled to digitally observe the Skagit Watershed, powered by HydroShare. <br />
<img src= "http://www.sauk-suiattle.com/images/Elliott.jpg"
style="float:left;width:175px;padding:20px">   
<br />
This is a beta version of a computational network-based sediment model was developed in order to connect processes of sediment supply on hillslopes, routing in streams, and deposition in reservoirs. The sediment model is developed in a framework called Landlab and driven by a physically-based, distributed hydrology model called DHSVM. The coupled sediment-hydrology model is designed to integrate relevant temporal and spatial scales of hillslope geomorphology, hydroclimatology and river network processes along with answering questions that are relevant to engineering application. The coupled model framework is designed to be applicable in other global watersheds, and could be useful for predicting sediment budgets particularly in the face of environmental and land use/land cover changes. 
<br /> <img src="https://www.washington.edu/brand/files/2014/09/W-Logo_Purple_Hex.png" style="float:right;width:120px;padding:20px">  
<br />
#### A Watershed Dynamics Model by the Watershed Dynamics Research Group in the Civil and Environmental Engineering Department at the University of Washington 

[Erkan To DO - -  add citations to Claire's AGU presentations and DOE report, Landlab paper, DHSVM-PNNL repo, others?]

## To run this notebook:

Click in each shaded cell below and "shift + enter" to run each code block. Alternatively, you can run groups of cells by clicking "Cell" on the menu above and selecting your run options from the pull-down menu. This is also where you can clear outputs from previous runs.

If an error occurs, click on *Kernal* and *Restart and Clear Outputs* in the menu above.

## 1. Introduction
Sediment discharge, its frequency and magnitude, and the size distribution of sediment load vary in space and time in a river. Sediment input into channels can be dominated by a single source such as a stratovolcano (e.g., Glacier Peak), or can be from distributed sources on hillslopes and zero-order basins, as well as from the banks of the channel itself through bank erosion.  Sediment is either transported within the flow as suspended load or transported along the riverbed as bedload. Suspended sediment includes sediments that are transported in suspension as a result of flow shear stress. It is composed of wash load (mostly clay and silt that is more or less continually in suspension) and suspension from bedload during higher flow; bed material in suspension includes coarse silt and sand that may move as bed load during lower flow. Bedload is generally assumed to represent 10% (or more generally in the 5% to 20% range) of the total sediment load. This percentage also depends on the location in the channel network and the type of watershed soil type. The suspended sediment load varies with discharge, often increasing nonlinearly as discharge rises because the greater turbulence associated with higher discharge allows a greater load of sediment to be held in suspension. 

<br /> <img src="https://www.hydroshare.org/resource/d5f1f96849554e03aa98bed5aff36cf7/data/contents/SedimentConcentrationbyVolume_Jakob_Jordon_2001.JPG" style="float:center;width:700px;padding:40px">

Figure 1. Classification of sediment flow processes with respect to sediment concentration, velocity, and process domains (modified from Jakob and Jordan, 2001).

Two related questions of interest for predicting erosion rates and sediment yields in the Puget Sound region are:  

1)	Do rivers that drain stratovolcanos and retreating glaciers exhibit larger erosion rates, and how modern and long-term erosion rates compare? 

2)	Can a single suspendend sediment transport relationship predict loads across Puget Sound basins?

In stratovolcanoes in high Alpine areas like Glacier Peak, a large portion of the loose pyroclastic deposits (clastic rock formed by volcanic explosion or ejected from a volcanic vent) are eroded by glaciers. Currently there are over a dozen glaciers on the sides of this volcano. In the model developed in this Notebook, we highlight network sediment results from seven glaciers; five in the Headwater Suiattle River (HUC 171100060201) including Dusty, Chocolate, Cool, Suiattle, and Honeycomb Glaciers and two in the Miners Creek Suiattle River (HUC 171100060202) including Vista and Ermine Glaciers. 

## 2.0 Methods

### 2.1. Install Python packages and libraries 
The CUAHSI JupyterHub server provides many Python packages and libraries, but to add additional libraries to your personal user space, use the cell below.  To request an Installation to the server, visit https://github.com/hydroshare/hydroshare-jupyterhub, create a New Issue, and add the label 'Installation Request'. Uncomment the lines below to install the library. Python2 and Python3 kernels are both available. If you are a new Landlab user on Hydroshare you are advised to run the code block below in your first run of this tutorial, then comment them out.

In [1]:
#%% Import modules and define functions
import os
import csv
import pandas as pd
import datetime
import time
import matplotlib.pyplot as plt
import numpy as np
import pickle
import scipy.stats

#Import utilities for importing and exporting to HydroShare
from utilities import hydroshare
import ogh

In [2]:
hs=hydroshare.hydroshare()
homedir = ogh.mapContentFolder(str(os.environ["HS_RES_ID"]))

print('Data will be loaded from and saved to:'+homedir)
os.chdir('/home/jovyan/work/notebooks/data/d5f1f96849554e03aa98bed5aff36cf7/d5f1f96849554e03aa98bed5aff36cf7/data/contents')
!pwd

Adding the following system variables:
   HS_USR_NAME = ChristinaBandaragoda
   HS_RES_ID = 87dc5742cf164126a11ff45c3307fd9d
   HS_RES_TYPE = compositeresource
   JUPYTER_HUB_IP = jupyter.cuahsi.org

These can be accessed using the following command: 
   os.environ[key]

   (e.g.)
   os.environ["HS_USR_NAME"]  => ChristinaBandaragoda
Successfully established a connection with HydroShare
Data will be loaded from and saved to:/home/jovyan/work/notebooks/data/87dc5742cf164126a11ff45c3307fd9d/87dc5742cf164126a11ff45c3307fd9d/data/contents
/home/jovyan/work/notebooks/data/d5f1f96849554e03aa98bed5aff36cf7/d5f1f96849554e03aa98bed5aff36cf7/data/contents


#### Run following cells to import extract and copy DHSVM temperature model Outflow.Only files on HydroShare

In [3]:
#hs.getResourceFromHydroShare('f16bdb504c6a4fb39e80ff1070a86704')

In [4]:
#!gunzip /home/jovyan/work/notebooks/data/f16bdb504c6a4fb39e80ff1070a86704/f16bdb504c6a4fb39e80ff1070a86704/data/contents/*/*

In [5]:
# DHSVM model output  - flow at each stream node
streamflow_input='Outflow.Only'

### 2.2 Import DHSVM network files and modeled streamflow 

In [6]:
# GIS file developed from https://github.com/pnnl/DHSVM-PNNL/tree/master/CreateStreamNetwork_PythonV
# Note for Zhuoron: This code needs it's own citation and reference
network_input = os.path.join(homedir, 'streamfile_2km2_sauk.txt')
network_input ='streamfile_2km2_sauk.txt'

In [7]:
!pwd

/home/jovyan/work/notebooks/data/d5f1f96849554e03aa98bed5aff36cf7/d5f1f96849554e03aa98bed5aff36cf7/data/contents


### 2.3 Set Model Parameters 

#### Universal Constants

In [8]:
g=9.81                     # acceleration of gravity (m/s2)
rho_w=1000                 # density of water (kg/m3)
v=10**-6                   # kinematic viscosity of water (m2/s)
sg=2.65                    # specific gravity 
bd_fine=1.13                # bulk density of fine sediments in reservoir 
bd_coarse=1.71              # bulk density of coarse sediments in reservoir 

#### Hydraulic geometry parameters

In [9]:
## hydraulic geometry (HG) parameters (note: a is constant, b is exponent)
ref_stream= 178 # DHSVM stream number where observations are collected that is used to scale other streams. For Elwha, this is the Lake Mills stream.

# stream width @ ref stream
w_const_ref=41.6
## stream depth HG parameters @ ref stream
a_d_ref=0.24
b_d_ref=0.41
### stream velocity HG parameters @ ref stream
a_u_ref=0.10
b_u_ref=0.58

## stream width HG upstream exponent
exp_w_us=0.5
## stream depth HG upstream exponent
exp_d_us=0.4
## stream velocity HG upstream exponent
exp_u_us=0.1

# roughness parameterization
a_n=1.08
b_n=-0.44

#### Geomorphology parameters

In [10]:
Drate=0.2/365.25 # denudation rate [mm/day]- based on computations/literature
beta_mw=10*365.25 # lag time [days] between mass wasting events- exponenetial distribution parameter
init_depth=1 # Set initial depth of sediment in channels [m] 
abrasion_alpha=0.027
abrasion_alpha_Fg=0.027
mw_pcnt_g=0.6 # percent of gravel in mass wasting deposit
mw_pcnt_s=1-mw_pcnt_g # percent of sand in mass wasting deposit

#### Bedload sediment transport parameters

In [11]:
## sediment transport parameterization per Wilcock and Crowe, 2003
# Channel grain size properties at Lake mills gage stream
Ch_Fs_LM=0.37
Ch_Fg_LM=1-Ch_Fs_LM
d90ch_LM=0.0275 # m -  d90 of channel bed= 27.5 mm=0.0275 m
dsandch_LM=0.00093 # m- d_sand of channel bed= 0.93 mm = 0.00093 m
dgravelch_LM=0.0132 # m- d_gravel of channel bed= 13.2 mm = 0.0132 m
dmeanch_LM=0.0125 # m- d_mean of channel bed= 12.5 mm = 0.0125 m

# Wilcock- Crowe Equation Parameters
A_gravel=14
chi_gravel=0.894
exp_gravel=0.5
phi_prime_gravel=1.35

A_sand=14
chi_sand=0.894
exp_sand=0.5
phi_prime_sand=1.35

#### Suspended sediment parameters

In [12]:
# Suspended Sediment Equation Parameters
a_s=1.17*10**-4 # regression coefficient from Curran et al., 2009
b_s_c=3        # regression coefficient for SS concentration from Curran et al., 2009
b_s_l=4        # regression coefficient for SS load from Curran et al., 2009
cf_s=1.07      # log-regression correction factor from Curran et al., 2009
K_ss=0.0864       # unit conversion factor from Curran et al., 2009  


# Network suspended sediment parametrization
# Key references: Patil et al., 2012
# for sand equations
c0_ss=1.1038
c1_ss=2.6626
c2_ss=5.6497
c3_ss=0.3822
c4_ss=-0.6174
c5_ss=0.1315
c6_ss=-0.0091

# for silt equations
tau_c_fines=0.015*(bd_fine-1)
a_w_m=0.08
n_w_m=1.65
b_w_m=3.5
m_w_m=1.88
c1_m=0.15
c2_m=b_w_m/((2*m_w_m-1)**0.5)


### 2.4 Code Model Functions

In [13]:
def find_nearest(array,value):
    val = (np.abs(array-value)).argmin()
    return array[val]

# Upload observed data
def create_q_obs_df(file_name, drainage_area):
    q=pd.read_excel(file_name, sheetname='data', skiprows=[0], header=None, usecols='A:D')
    q.columns=['year','month','day','flow_cfs']
    q_dates=pd.to_datetime(q.loc[:,['year','month','day']])
    q.set_index(q_dates, inplace=True)
    q.drop(['year','month','day'],axis=1, inplace=True)
    q_cms=q.flow_cfs/(3.28084**3)
    q_mmday=q_cms*1000*3600*24/drainage_area
    q=pd.concat([q_cms, q, q_mmday],axis=1)
    q.columns=['flow_cms','flow_cfs', 'flow_mmday']
    return q

def import_obs_folders(obs_folder,streamflow_obs_input_txt):
    os.chdir(obs_folder)
    LM_usgs= np.genfromtxt(streamflow_obs_input_txt,skip_header=1,dtype=str)                   
    n_1=len(LM_usgs[:,0]) # n is number of days in the record
    date_LM=np.full(n_1,'', dtype=object) # Preallocate date_1 matrix
    for x in range(0,n_1): # Cycle through all days of the year
        date_LM_temp=datetime.date(int(LM_usgs[x,0]),int(LM_usgs[x,1]),
                                               int(LM_usgs[x,2]))
        # make numpy array of individual temporary datetime objects
        date_LM[x]=date_LM_temp # enter temporary object into preallocated date matrix
    del(date_LM_temp) # delete temporay object
    
    # Extract remaining variables and convert to standard units:
    Q_day=np.array((LM_usgs[:,3]), dtype='float64')
    Q_day=Q_day/(3.28084**3) # convert from ft^3/s to m^3/s
    SSC_day=np.array(LM_usgs[:,7], dtype='float64')
    SSL_day=np.array(LM_usgs[:,9], dtype='float64')
    T_day=np.array(LM_usgs[:,11], dtype='float64')
    
    LM_data=pd.DataFrame({"Q_m3s": Q_day, "SSC_mgL": SSC_day,
                          "SSL_tonsday":SSL_day, "T_fnu":T_day},index=pd.to_datetime(date_LM))
    
    # Now find the 50 % exceedance flow at Lakem Mills gage- use complete water years only
    Q_curve=LM_data.loc[datetime.date(1994, 10, 1):datetime.date(1997, 9, 30), 'Q_m3s'].values
    Q_curve=np.append(Q_curve,LM_data.loc[datetime.date(2004, 10, 1):datetime.date(2011, 9, 30), 'Q_m3s'].values)
    
    # FUTURE:  Use calibrated DHSVM outputs instead?
    Q_RC=np.sort(Q_curve)
    n_curve=len(Q_curve)
    ep_Q_curve=np.array(range(1,n_curve+1))/(n_curve+1)
    cum_pcntl=ep_Q_curve
    Q_LM_50EP=Q_RC[np.where(cum_pcntl==find_nearest(cum_pcntl, 0.50))] # 50th percentile
            
    return (LM_data, Q_LM_50EP)

def compute_NSE_rs (modeled, observed):
    NSE=1-((np.sum((modeled-observed)**2))/(np.sum((modeled-np.mean(observed))**2)))
    WC,WC_0,WC_r, WC_p, WC_err=scipy.stats.linregress(modeled, observed)
    r2=WC_r**2
    print('r2=',r2)
    print('NSE=',NSE)
    return NSE, r2


def setup_network(model_input_folder):                      
    #os.chdir(model_input_folder)  
    network=pd.read_table(network_input, delimiter='\t', index_col=0,\
                          usecols=[0, 1, 2, 3, 4, 5, 6, 7])
    network.columns=['segment_length_m','local_ca','dest_channel_id',\
                     'segment_slope','total_ca_mean','segment_order',\
                     'channel_class_id'] 
    return (network)

#network=setup_network(model_input_folder)

def run_stochastic_mass_wasting(ref_stream, a_u_ref, b_u_ref, a_d_ref, b_d_ref, a_n, b_n, ng_obs_bar, S, total_ca_ref, total_ca, Qref, Q):  
    rho_w=1000
    # ref stream values for given flow
    Uref=a_u_ref*Qref**b_u_ref
    Dref=a_d_ref*Qref**b_d_ref
    # stream-of-interest values for given flow
    U=Uref*(total_ca**exp_w_us)/(total_ca_ref**exp_w_us)
    D=Dref*(total_ca**exp_d_us)/(total_ca_ref**exp_d_us)
    ng=ng_obs_bar*a_n*D**b_n
    tau=rho_w*g*((ng*U)**(3/2))*S**(1/4)
    u=(tau/rho_w)**0.5
    return tau, u         

def compute_channel_properties(ref_stream, a_u_ref, b_u_ref, a_d_ref, b_d_ref, a_n, b_n, ng_obs_bar, S, total_ca_ref, total_ca, Qref, Q):  
    rho_w=1000
    # ref stream values for given flow
    Uref=a_u_ref*Qref**b_u_ref
    Dref=a_d_ref*Qref**b_d_ref
    # stream-of-interest values for given flow
    U=Uref*(total_ca**exp_u_us)/(total_ca_ref**exp_u_us)
    D=Dref*(total_ca**exp_d_us)/(total_ca_ref**exp_d_us)
    ng=ng_obs_bar*a_n*D**b_n
    tau=rho_w*g*((ng*U)**(3/2))*S**(1/4)
    u_star=(tau/rho_w)**0.5
    return tau, u_star         
        
def run_wc2003_2F_model (tau, tau_r_sand,tau_r_gravel):
    # Constants
    A=14
    chi=0.894
    exp=0.5
    phi_prime=1.35 
             
    # Run WC 2003 Two-Fraction Model
    phi_gravel=tau/tau_r_gravel
    phi_sand=tau/tau_r_sand
    
    if phi_gravel<phi_prime:
        Wstar_gravel=0.002*(phi_gravel)**7.5
    elif chi/((phi_gravel)**exp)>=1: # Checka that term in paraentheses is not negative
       Wstar_gravel=0
    else:
        Wstar_gravel=A*((1-(chi/((phi_gravel)**exp)))**(4.5))

    if phi_sand<phi_prime:
        Wstar_sand=0.002*(phi_sand)**7.5
    elif chi/((phi_sand)**exp)>=1: # Checka that term in paraentheses is not negative
       Wstar_sand=0
    else:
        Wstar_sand=A*((1-(chi/((phi_sand)**exp)))**(4.5))        
    return (Wstar_gravel, Wstar_sand)    

### 2.5 Create arrays of streamflow forcings

#### Set sediment model run time = length of modeled streamflow input file output.Only

In [14]:
forcing_start_date=datetime.date(1915,1,1)
forcing_end_date=datetime.date(2011,1,1)



In [15]:
!pwd

/home/jovyan/work/notebooks/data/d5f1f96849554e03aa98bed5aff36cf7/d5f1f96849554e03aa98bed5aff36cf7/data/contents


In [16]:
#List of all links in network for running sediment model
stream_columns=['Date',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439]
   
forcing_date_range=pd.date_range(forcing_start_date, forcing_end_date)
streamflow=pd.DataFrame(index=forcing_date_range, columns=stream_columns)
#streamflow.loc[forcing_date_range[0],:]=pd.read_table(streamflow_input, skiprows=[0,1], header=None,sep='\s+')
streamflow=pd.read_table(streamflow_input, skiprows=[0,1], header=None,sep='\s+')
streamflow.columns=stream_columns        
streamflow=streamflow.set_index('Date', drop=True)
streamflow.index = pd.to_datetime(streamflow.index)
streamflow
type(streamflow.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [17]:

Qmod_median=streamflow.median(axis=0)
pickle.dump(streamflow, open("streamflow.py", "wb"))
pickle.dump(Qmod_median, open("Qmod_median.py", "wb"))

In [18]:
#List to run sediment in Headwaters Suiattle HUC12 basin only  - extracted from ArcGIS map SaukSediment.mxd 

#headstream_columns=[260,270,276,298,300,301,316,317,318,323,325,329,333,339,340,341,342,345,349,351,352,368,371]
headstream_columns=stream_columns
headstream=streamflow.loc[:,headstream_columns]
Qmod_median=headstream.median(axis=0)
pickle.dump(headstream, open("streamflow.json", "wb"))
pickle.dump(Qmod_median, open("Qmod_median.json", "wb"))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [19]:
print(Qmod_median)

Date         NaN
1       13.19130
2       13.16050
3        0.02720
4       13.13100
5       13.06350
6        0.06320
7        0.01880
8        0.03460
9        0.01220
10       0.11910
11       0.05170
12       0.02060
13       0.02790
14       0.04000
15       0.01370
16       0.01510
17       0.01350
18       0.11210
19      12.94290
20       0.03210
21       0.08670
22       0.02490
23       0.14680
24      12.85250
25      12.69420
26       0.01840
27       0.02440
28       0.01650
29       0.04040
          ...   
410      0.01310
411      0.09900
412      0.19530
413      0.20040
414      0.03450
415      0.04710
416      0.02550
417      0.02250
418      0.03160
419      0.25190
420      0.05110
421      0.37340
422      0.15870
423      0.16745
424      0.01810
425      0.08980
426      0.10950
427      0.01910
428      0.07610
429      0.01750
430      0.05460
431      0.03200
432      0.08390
433      0.16820
434      0.00980
435      0.04440
436      0.04060
437      0.052

In [20]:
ref_stream=260

### 2.6 Create sediment model network file

In [21]:
network=setup_network(homedir)
# Downstream links and distances
ds=pd.DataFrame(index=network.index, columns=['ds_strms','ds_dist_array','ds_dist_m']) # establish array of downstream stream numbers
strm_orders_rev=np.unique(network.segment_order)[::-1] # get all segement orders in the watershed and sort from highest to lowest
for st_o in strm_orders_rev:
    for i in network.index:
        if network.segment_order.loc[i]==st_o: # go in order of stream orders
            j=network.loc[i, 'dest_channel_id']
            if j in network.index:
                ds.loc[i,'ds_strms']=np.append(j, ds.loc[j,'ds_strms'])
                ds.loc[i,'ds_dist_array']=np.append(network.loc[j, 'segment_length_m'], ds.loc[j,'ds_strms'])
                ds.loc[i,'ds_dist_m']=np.nansum(ds.loc[i,'ds_dist_array'])
            else:
                ds.loc[i,'ds_strms']=[]
                ds.loc[i,'ds_dist_array']=[]
                ds.loc[i,'ds_dist_m']=0
                
network=pd.concat([network, ds], axis=1, join_axes=[network.index]) # add these to network array 
network[['ds_dist_m']]=network[['ds_dist_m']].astype(float)
network = network.loc[:,~network.columns.duplicated()] # Ensure that there are no duplicate columns!
    
# Stream Width- assume constant
width=pd.DataFrame(index=network.index, columns=['width'],
                       data=w_const_ref*((network['total_ca_mean'].values)**exp_w_us)/(((network['total_ca_mean'][ref_stream])**exp_w_us)))
network=pd.concat([network, width],axis=1, join_axes=[network.index]) # add these to network array
    

In [22]:
#Use for full network
#strm_link_vals=network.index

#Use for smaller network
strm_link_vals=headstream_columns


In [23]:
!ls -lt


total 716336
-rw-r--r-- 1 jovyan users      6351 Jan 30 23:18 Qmod_median.json
-rw-r--r-- 1 jovyan users 124993055 Jan 30 23:18 streamflow.json
-rw-r--r-- 1 jovyan users      6330 Jan 30 23:18 Qmod_median.py
-rw-r--r-- 1 jovyan users 124709627 Jan 30 23:18 streamflow.py
-rwxrwx--- 1 root   users     86477 Jan 30 23:17 Sauk Sediment Headwaters Model 20190128.ipynb
-rw-r--r-- 1 jovyan users   1320039 Jan 30 23:17 Sauk Sediment Network Model 20190130.ipynb
-rw-r--r-- 1 jovyan users       264 Jan 30 23:16 tL_mw.npy
-rw-r--r-- 1 jovyan users       264 Jan 30 23:16 time_mw.npy
-rw-r--r-- 1 jovyan users     17928 Jan 30 23:16 Vm_t_annual.npy
-rw-r--r-- 1 jovyan users     17928 Jan 30 23:16 Vss_t_annual.npy
-rw-r--r-- 1 jovyan users     17928 Jan 30 23:16 Vsb_t_annual.npy
-rw-r--r-- 1 jovyan users     17928 Jan 30 23:16 Vs_t_annual.npy
-rw-r--r-- 1 jovyan users     17928 Jan 30 23:16 Vg_t_annual.npy
-rw-r--r-- 1 jovyan users     17928 Jan 30 23:16 Vs_cap_annual.npy
-rw-r--r-- 1 jovyan users   

In [24]:

Qmod_median[ref_stream]

0.79325000000000001

In [25]:
# Grain Size
#os.chdir(model_data_folder)
# strm_link_vals=pickle.load(open('strm_link_vals.py', 'rb')) # stream link values of interest
n_strms=len(strm_link_vals) # number of streams/tributary areas
strm_orders=np.unique(network.segment_order)
    
## Upstream grain size diameters from Downstream Fining Equation
d90_ch_us=d90ch_LM/np.exp(-abrasion_alpha*(network.ds_dist_m-network.ds_dist_m[ref_stream])/1000)
dsand_ch_us=dsandch_LM/np.exp(-abrasion_alpha*(network.ds_dist_m-network.ds_dist_m[ref_stream])/1000)
dgravel_ch_us=dgravelch_LM/np.exp(-abrasion_alpha*(network.ds_dist_m-network.ds_dist_m[ref_stream])/1000)
dmean_ch_us=dmeanch_LM/np.exp(-abrasion_alpha*(network.ds_dist_m-network.ds_dist_m[ref_stream])/1000)

Qmod_median=pickle.load(open('Qmod_median.json', 'rb'))
H_median_lm=a_d_ref*(Qmod_median[ref_stream])**b_d_ref   
H_median_all=H_median_lm*((network['total_ca_mean'].values)**exp_d_us)/(((network['total_ca_mean'][ref_stream])**exp_d_us))
ng_bar=(1/(np.sqrt(8*g)))*(H_median_all**(1/6))/(1.26-2.16*np.log10(d90_ch_us/H_median_all))
    
network=pd.concat([network,pd.DataFrame({"d90_ch_m": d90_ch_us,
                                          "dsand_ch_m": dsand_ch_us,
                                          "dgravel_ch_m": dgravel_ch_us,
                                          "dmean_ch_m": dmean_ch_us,
                                          "Ch_Fs_strms": Ch_Fs_LM, # start out with all the same as LM and will let evolve
                                          "ng_bar":ng_bar},index=strm_link_vals)], axis=1, join_axes=[network.index]) 
    
network=network.loc[strm_link_vals,:]
del(dsand_ch_us, dgravel_ch_us, dmean_ch_us)
    #os.chdir(model_data_folder)
pickle.dump(network, open("network.json", "wb"))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:25: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


### 2.7 Initialize model 

In [26]:
strm_link_vals=network.index # stream link values of interest
n_strms=len(strm_link_vals) # number of streams/tributary areas
strm_orders=np.unique(network.segment_order) # array of unique stream orders
start_year_len=len(streamflow.index[streamflow.index.year==streamflow.index[0].year]) # number of days in first year

# Preallocate sediment volume data frames representing sediment "buckets" at the end of each timestep (day)
# Vg=gravel; Vs=sand; Vm=mud (silt, clay)
# Volumes stored on bed [m3]
Vg_b=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,start_year_len+1))
Vs_b=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,start_year_len+1))
Vm_b=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,start_year_len+1))

# Volumes deposited from mass wasting event [m3]
Vg_mw=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,start_year_len+1))
Vs_mw=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,start_year_len+1))
Vm_mw=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,start_year_len+1))

# Volume of bedload that stream had capacity to transport  [m3]
Vg_cap=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,start_year_len+1))
Vs_cap=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,start_year_len+1))

# Volume transported out of stream  [m3]
Vg_t=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,start_year_len+1))
Vs_t=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,start_year_len+1))
Vsb_t=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,start_year_len+1))
Vss_t=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,start_year_len+1))
Vm_t=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,start_year_len+1))

# Preallocate sediment volume data frames representing sediment "buckets" at the end of each year
# Volumes stored on bed [m3]
Vg_b_annual=pd.DataFrame(0, index=strm_link_vals, columns=np.unique(streamflow.index.year))
Vs_b_annual=pd.DataFrame(0, index=strm_link_vals, columns=np.unique(streamflow.index.year))
Vm_b_annual=pd.DataFrame(0, index=strm_link_vals, columns=np.unique(streamflow.index.year))
# Volumes deposited from mass wasting event [m3]
Vg_mw_annual=pd.DataFrame(0, index=strm_link_vals, columns=np.unique(streamflow.index.year))
Vs_mw_annual=pd.DataFrame(0, index=strm_link_vals, columns=np.unique(streamflow.index.year))
Vm_mw_annual=pd.DataFrame(0, index=strm_link_vals, columns=np.unique(streamflow.index.year))
# Volume of bedload  that stream had capacity to transport  [m3]
Vg_cap_annual=pd.DataFrame(0, index=strm_link_vals, columns=np.unique(streamflow.index.year)) 
Vs_cap_annual=pd.DataFrame(0, index=strm_link_vals, columns=np.unique(streamflow.index.year))
# Volume transported out of stream  [m3]
Vg_t_annual=pd.DataFrame(0, index=strm_link_vals, columns=np.unique(streamflow.index.year))
Vs_t_annual=pd.DataFrame(0, index=strm_link_vals, columns=np.unique(streamflow.index.year))
Vsb_t_annual=pd.DataFrame(0, index=strm_link_vals, columns=np.unique(streamflow.index.year))
Vss_t_annual=pd.DataFrame(0, index=strm_link_vals, columns=np.unique(streamflow.index.year))
Vm_t_annual=pd.DataFrame(0, index=strm_link_vals, columns=np.unique(streamflow.index.year))

# Other- temporary
C_ss=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,start_year_len+1))
Vs_dep=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,start_year_len+1))

#Set any initial (time=0) conditions in streams
# Volumes stored on bed [m3]
Vg_b.loc[:,0]=0.33*init_depth*network.segment_length_m*network.width # set initial volume of sediment stored in channel
Vs_b.loc[:,0]=0.34*init_depth*network.segment_length_m*network.width # set initial volume of sediment stored in channel
Vg_b.loc[:,0]=0.33*init_depth*network.segment_length_m*network.width # set initial volume of sediment stored in channel
# Volumes deposited from mass wasting event [m3]
Vg_mw.loc[:,0]=0
Vs_mw.loc[:,0]=0
Vm_mw.loc[:,0]=0

# Volume that stream had capacity to transport  [m3]
Vg_cap.loc[:,0]=0
Vs_cap.loc[:,0]=0

# Volume transported out of stream  [m3]
Vg_t.loc[:,0]=0
Vs_t.loc[:,0]=0
Vm_t.loc[:,0]=0

# Set arrays for times between mass wasting events
time_mw=pd.DataFrame(0, index=strm_link_vals, columns=['output']) # time since last mass wasting event for each stream link
tL_mw=np.ceil(np.random.exponential(scale=beta_mw, size=len(strm_link_vals))) # time (years) until first mass wasting event- randomly sampled from exponential distribution
tL_mw=pd.DataFrame(data=tL_mw, index=strm_link_vals, columns=['output']) # turn into a data frame that will grow


# 3.0 Run Model 

In [ ]:
start = time.time()
date_string=time.strftime("%Y-%m-%d_%H%M")
os.mkdir(date_string)
day_of_year=0
## RUN MODEL
for i in range (0,len(streamflow.index)):
    print(streamflow.index[i])
    date=streamflow.index[i]
    day_of_year=day_of_year+1
    time_mw=time_mw+1 # start new year
   # run stochastic mass wasting generator for all stream links
    for st_o in strm_orders:
        for j in network.index.values:
            if network.segment_order.loc[j]==st_o: # go in order of stream orders
                if time_mw.loc[j,'output']==tL_mw.loc[j,'output']: # if time since the last mass wasting event is the same as randomly generated lag time
                    # Generate random values for percent gravel, sand, and mud,
                    temp_rand=np.random.rand(3)
                    mw_pcnts=temp_rand/sum(temp_rand)
                    # Generate mass wasting volumes based on denudation rate
                    Vg_mw.loc[j,day_of_year]=mw_pcnts[0]*Drate*network.loc[j,'local_ca']*(1/1000)*tL_mw.loc[j,'output']
                    Vs_mw.loc[j,day_of_year]=mw_pcnts[1]*Drate*network.loc[j,'local_ca']*(1/1000)*tL_mw.loc[j,'output']                   
                    Vm_mw.loc[j,day_of_year]=mw_pcnts[2]*Drate*network.loc[j,'local_ca']*(1/1000)*tL_mw.loc[j,'output']                   
                    # Update mass wasting clocks
                    time_mw.loc[j,'output']=0 # Reset counter until next mass wasting event
                    tL_mw.loc[j,'output']=np.ceil(np.random.exponential(scale=beta_mw))  # generate a new lag time until next mass wasting event and add it to array of lag times#
                else: # If have not reach the lag time for a mass wasting event at the stream then the volume will be the same as last year
                    # Not mass wasting volume is added
                    Vg_mw.loc[j,day_of_year]=0
                    Vs_mw.loc[j,day_of_year]=0
                    Vm_mw.loc[j,day_of_year]=0

                # Add mass wasting volume to the bed from previous timestep
                Vg_b.loc[j,day_of_year]=Vg_b.loc[j,day_of_year-1]+Vg_mw.loc[j,day_of_year]
                Vs_b.loc[j,day_of_year]=Vs_b.loc[j,day_of_year-1]+Vs_mw.loc[j,day_of_year] 
                Vm_b.loc[j,day_of_year]=Vm_b.loc[j,day_of_year-1]+Vm_mw.loc[j,day_of_year] 

                # Add in sediment from upstream tributaries
                if st_o!=1: # if not at a headwater stream (i.e., stream order >1), need to add volume of sediment coming from upstream
                    feeder_links=network.loc[network['dest_channel_id']==j].index.values # find the stream links that feed into the link which are immediately upstream
                    # sum up the volume transported out of the upstream link and add to current volume
                    Vg_b.loc[j,day_of_year]=Vg_b.loc[j,day_of_year]+np.nansum(Vg_t.loc[feeder_links,day_of_year])  
                    Vs_b.loc[j,day_of_year]=Vs_b.loc[j,day_of_year]+np.nansum(Vs_t.loc[feeder_links,day_of_year])
                    Vm_b.loc[j,day_of_year]=Vm_b.loc[j,day_of_year]+np.nansum(Vm_t.loc[feeder_links,day_of_year])
                
                # Compute shear stress and shear velocity from modeled flow
                Q=streamflow.loc[date,j]
                Qvol=Q*3600*24 # m3 of streamflow throughout the day [m3]
                tau_strm, u_strm= compute_channel_properties(ref_stream, a_u_ref, b_u_ref, a_d_ref, b_d_ref, a_n, b_n, network['ng_bar'][j], 
                                                             network.segment_slope[j], network['total_ca_mean'][ref_stream], network['total_ca_mean'][j], 
                                                             streamflow.loc[date,ref_stream], Q)

                # Suspended Sediment
                C_ss.loc[j,day_of_year]=(Vs_b.loc[j,day_of_year]+Vm_b.loc[j,day_of_year])/Qvol # Concentration of suspended sediment (unitless)
                
                # Suspended sediment- silt: Compute deposition and transport
                if tau_strm>tau_c_fines:
                    Vm_d=0
                else: 
                    if C_ss.loc[j,day_of_year]<=c1_m:
                        v_ss_m=a_w_m*(c1_m**n_w_m)/((c1_m**2+b_w_m**2)**m_w_m)
                    if C_ss.loc[j,day_of_year]>c2_m:
                        v_ss_m=a_w_m*(c2_m**n_w_m)/((c2_m**2+b_w_m**2)**m_w_m)
                    else:
                        v_ss_m=a_w_m*(C_ss.loc[j,day_of_year]**n_w_m)/((C_ss.loc[j,day_of_year]**2+b_w_m**2)**m_w_m)                        
                    Vm_d=(1-(tau_strm/tau_c_fines))*v_ss_m*C_ss.loc[j,day_of_year]*network.loc[j,'segment_length_m']*network.loc[j,'width']*3600*24
                Vm_t.loc[j,day_of_year]=Vm_b.loc[j,day_of_year]-min(Vm_d, Vm_b.loc[j,day_of_year])
                Vm_b.loc[j,day_of_year]=max(Vm_b.loc[j,day_of_year]-Vm_t.loc[j,day_of_year],0)
                
                # Suspended sediment- sand: Compute deposition/erosion
                #v_ss_s=g*(network.loc[j,'dsand_ch_m']**2)*(sg-1)/(18*v) # settling velocity of sand
                d_star_ss=(((sg-1)*g*network.loc[j,'dsand_ch_m']**3)/(v**2))**(1/3)
                v_ss_s=(v/network.loc[j,'dsand_ch_m'])*(np.sqrt((1/4)*(24/1.5)**(2/1)+((4*d_star_ss**3)/(3*1.5))**(1/1))-(1/2)*(24/1.5)**(1/1))**1                
                Z_R=v_ss_s/(0.41*u_strm)
                int_Z_R=1/(c0_ss+c1_ss*Z_R+c2_ss*Z_R**2+c3_ss*Z_R**3+c4_ss*Z_R**4+c5_ss*Z_R**5+c6_ss*Z_R**6)
                Vs_dep_comp=v_ss_s*C_ss.loc[j,day_of_year]/int_Z_R*network.loc[j,'segment_length_m']*network.loc[j,'width']*3600*24 #m3/day- volume of deposited sand
                if Vs_dep_comp<0:
                    Vs_dep.loc[j,day_of_year]=0
                elif Vs_dep_comp>Vs_b.loc[j,day_of_year]:
                    Vs_dep.loc[j,day_of_year]=Vs_b.loc[j,day_of_year]
                else:
                    Vs_dep.loc[j,day_of_year]=Vs_dep_comp

                # Sand transported out as suspended sediment
                Vss_t.loc[j,day_of_year]=Vs_b.loc[j,day_of_year]-Vs_dep.loc[j,day_of_year]
                
                # Sand remaining on bed, with potential to be transferred as bedload
                Vs_b.loc[j,day_of_year]=Vs_dep.loc[j,day_of_year]
                
                # Recompute W&C 2003 parameterizaion of sand and gravel on the channel bed
                network.loc[j,'Ch_Fs_strms']=Vs_b.loc[j,day_of_year]/(Vs_b.loc[j,day_of_year]+Vg_b.loc[j,day_of_year])
                tau_star_rsm=0.021+0.015*np.exp(-20*network.loc[j,'Ch_Fs_strms']) # dimensionless reference shear stress for mean grain size
                tau_rsm=tau_star_rsm*(sg-1)*rho_w*g*(network.loc[j,'dmean_ch_m']) # reference shear stress for mean grain size [N/m2]
                b_sand=0.67/(1+np.exp(1.5-(network.loc[j,'dsand_ch_m']/network.loc[j,'dmean_ch_m']))) # b parameter for sand
                b_gravel=0.67/(1+np.exp(1.5-(network.loc[j,'dgravel_ch_m']/network.loc[j,'dmean_ch_m']))) # b parameter for gravel
                tau_r_sand=tau_rsm*(network.loc[j,'dsand_ch_m']/network.loc[j,'dmean_ch_m'])**b_sand # reference tau for sand [N/m2]
                tau_r_gravel=tau_rsm*(network.loc[j,'dgravel_ch_m']/network.loc[j,'dmean_ch_m'])**b_gravel # reference tau for gravel [N/m2]
                tau_star_r_sand=tau_r_sand/(rho_w*g*(sg-1)*network.loc[j,'dsand_ch_m'])
                tau_star_r_gravel=tau_r_gravel/(rho_w*g*(sg-1)*network.loc[j,'dgravel_ch_m'])
                         
                # Compute Bedload sediment transport capacity with calibrated Wilcock and Crowe equation
                Wstar_gravel, Wstar_sand=run_wc2003_2F_model (tau_strm, tau_star_r_sand, tau_star_r_gravel)
                
                # Compute gravel transport capacity, volume transported, and volume remaining
                Vg_cap.loc[j,day_of_year]=3600*24*((u_strm)**3)*Wstar_gravel*(1-network.loc[j,'Ch_Fs_strms'])/((sg-1)*g)*network.loc[j,'width'] #  m3 (unit is m3/day; timestep is 1 day)
                Vg_t.loc[j,day_of_year]=np.min([Vg_cap.loc[j,day_of_year],Vg_b.loc[j,day_of_year]]) # gravel: volume transported is the minimum between available volume and capacity of flow
                Vg_b.loc[j,day_of_year]=Vg_b.loc[j,day_of_year]-Vg_t.loc[j,day_of_year] # new volume deposited in stream- difference between existing volume and volume transported

                # Compute sand bedload transport capacity, volume transported, and volume remaining
                Vs_cap.loc[j,day_of_year]=3600*24*((u_strm)**3)*Wstar_sand*(network.loc[j,'Ch_Fs_strms'])/((sg-1)*g)*network.loc[j,'width'] # m3 (unit is m3/day; timestep is 1 day)
                Vsb_t.loc[j,day_of_year]=np.min([Vs_cap.loc[j,day_of_year],Vs_b.loc[j,day_of_year]]) # sand: volume transported is the minimum between available volume and capacity of flow
                Vs_t.loc[j,day_of_year]=Vsb_t.loc[j,day_of_year]+Vss_t.loc[j,day_of_year] # add volumes of sand transported as bedload and suspended load 
                Vs_b.loc[j,day_of_year]=Vs_b.loc[j,day_of_year]-Vsb_t.loc[j,day_of_year] # new volume deposited in stream- difference between existing volume and volume transported                             
                
    if date.month==12 and date.day==31:
        if date.year==streamflow.index[streamflow.index.year==streamflow.index[-1].year][0].year:
            next_year_len=0
        else: next_year_len=len(streamflow.index[streamflow.index.year==date.year+1])
        
        Vg_b_last=Vg_b.loc[:, Vg_b.columns[-1]]
        Vs_b_last=Vs_b.loc[:, Vs_b.columns[-1]]
        Vm_b_last=Vm_b.loc[:, Vm_b.columns[-1]]
        Vg_mw_last=Vg_mw.loc[:, Vg_mw.columns[-1]]
        Vs_mw_last=Vs_mw.loc[:, Vs_mw.columns[-1]]
        Vm_mw_last=Vm_mw.loc[:, Vm_mw.columns[-1]]
        Vg_cap_last=Vg_cap.loc[:, Vg_cap.columns[-1]]
        Vs_cap_last=Vs_cap.loc[:, Vs_cap.columns[-1]]
        Vg_t_last=Vg_t.loc[:, Vg_t.columns[-1]]
        Vs_t_last=Vs_t.loc[:, Vs_t.columns[-1]]
        Vsb_t_last=Vsb_t.loc[:, Vsb_t.columns[-1]]
        Vss_t_last=Vss_t.loc[:, Vss_t.columns[-1]]
        Vm_t_last=Vm_t.loc[:, Vm_t.columns[-1]]
        C_ss_last=C_ss.loc[:, C_ss.columns[-1]]
        Vs_dep_last=Vs_dep.loc[:, C_ss.columns[-1]]

        Vg_b_annual.loc[:, date.year]=Vg_b_last
        Vs_b_annual.loc[:, date.year]=Vs_b_last
        Vg_mw_annual.loc[:, date.year]=Vg_mw[1::].sum(axis=1)
        Vs_mw_annual.loc[:, date.year]=Vs_mw[1::].sum(axis=1)
        Vg_cap_annual.loc[:, date.year]=Vg_cap[1::].sum(axis=1)
        Vs_cap_annual.loc[:, date.year]=Vs_cap[1::].sum(axis=1)
        Vg_t_annual.loc[:, date.year]=Vg_t[1::].sum(axis=1)
        Vs_t_annual.loc[:, date.year]=Vs_t[1::].sum(axis=1)
        Vsb_t_annual.loc[:, date.year]=Vsb_t[1::].sum(axis=1)
        Vss_t_annual.loc[:, date.year]=Vss_t[1::].sum(axis=1)
        Vm_t_annual.loc[:, date.year]=Vm_t[1::].sum(axis=1)
        
        np.save('Vg_b_annual', Vg_b_annual)
        np.save('Vs_b_annual', Vs_b_annual)
        np.save('Vg_mw_annual', Vg_mw_annual)   
        np.save('Vs_mw_annual', Vs_mw_annual)   
        np.save('Vg_cap_annual', Vg_cap_annual)
        np.save('Vs_cap_annual', Vs_cap_annual)
        np.save('Vg_t_annual', Vg_t_annual)
        np.save('Vs_t_annual', Vs_t_annual)
        np.save('Vsb_t_annual', Vsb_t_annual)
        np.save('Vss_t_annual', Vss_t_annual)
        np.save('Vm_t_annual', Vm_t_annual)
        np.save('time_mw', time_mw)
        np.save('tL_mw', tL_mw)
                
        Vg_b=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,next_year_len+1))
        Vs_b=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,next_year_len+1))
        Vg_mw=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,next_year_len+1))
        Vs_mw=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,next_year_len+1))
        Vg_cap=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,next_year_len+1))
        Vs_cap=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,next_year_len+1))
        Vg_t=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,next_year_len+1))
        Vs_t=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,next_year_len+1))
        Vsb_t=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,next_year_len+1))
        Vss_t=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,next_year_len+1))
        Vm_t=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,next_year_len+1))
        C_ss=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,next_year_len+1)) 
        Vs_dep=pd.DataFrame(0, index=strm_link_vals, columns=np.arange(0,next_year_len+1)) 

        Vg_b.loc[:, Vg_b.columns[0]]=Vg_b_last
        Vs_b.loc[:, Vs_b.columns[0]]=Vs_b_last
        Vg_mw.loc[:, Vg_mw.columns[0]]=Vg_mw_last
        Vs_mw.loc[:, Vs_mw.columns[0]]=Vs_mw_last
        Vg_cap.loc[:, Vg_cap.columns[0]]=Vg_cap_last
        Vs_cap.loc[:, Vs_cap.columns[0]]=Vs_cap_last
        Vg_t.loc[:, Vg_t.columns[0]]=Vg_t_last
        Vs_t.loc[:, Vs_t.columns[0]]=Vs_t_last
        Vsb_t.loc[:, Vsb_t.columns[0]]=Vsb_t_last
        Vss_t.loc[:, Vss_t.columns[0]]=Vss_t_last
        Vm_t.loc[:, Vm_t.columns[0]]=Vm_t_last
        C_ss.loc[:, C_ss.columns[0]]=C_ss_last
        Vs_dep.loc[:, Vs_dep.columns[0]]=Vs_dep_last
        
        day_of_year=0



# Save results!
end = time.time()
print(end - start)

basin_flag='Sauk'
time_flag='2000'
routing_method='wc'
#network_output = os.path.join(homedir, date_string)
network_output = os.path.join(date_string)
os.chdir(network_output) 

np.save('Vg_b_annual', Vg_b_annual)
np.save('Vs_b_annual', Vs_b_annual)
np.save('Vg_mw_annual', Vg_mw_annual)
np.save('Vs_mw_annual', Vs_mw_annual)
np.save('Vg_cap_annual', Vg_cap_annual)
np.save('Vs_cap_annual', Vs_cap_annual)
np.save('Vg_t_annual', Vg_t_annual)
np.save('Vs_t_annual', Vs_t_annual)
np.save('Vsb_t_annual', Vsb_t_annual)
np.save('Vss_t_annual', Vss_t_annual)
np.save('Vm_t_annual', Vm_t_annual)
np.save('Vs_dep', Vs_dep)
np.save('C_ss',  C_ss)
np.save('time_mw', time_mw)
np.save('tL_mw', tL_mw)


Vg_b_annual.to_csv(str(basin_flag + '_' + time_flag + '_' + routing_method  + '_Vg_b_annual.csv'),index=False,header=None,sep=',')
Vs_b_annual.to_csv(str(basin_flag + '_' + time_flag + '_' + routing_method  + '_Vs_b_annual.csv'),index=False,header=None,sep=',')
Vg_mw_annual.to_csv(str(basin_flag + '_' + time_flag + '_' + routing_method  + '_Vg_mw_annual.csv'),index=False,header=None,sep=',')
Vs_mw_annual.to_csv(str(basin_flag + '_' + time_flag + '_' + routing_method  + '_Vs_mw_annual.csv'),index=False,header=None,sep=',')
Vg_cap_annual.to_csv(str(basin_flag + '_' + time_flag + '_' + routing_method  + '_Vg_cap_annual.csv'),index=False,header=None,sep=',')
Vs_cap_annual.to_csv(str(basin_flag + '_' + time_flag + '_' + routing_method  + '_Vs_cap_annual.csv'),index=False,header=None,sep=',')
Vg_t_annual.to_csv(str(basin_flag + '_' + time_flag + '_' + routing_method  + '_Vg_t_annual.csv'),index=False,header=None,sep=',')
Vs_t_annual.to_csv(str(basin_flag + '_' + time_flag + '_' + routing_method  + '_Vs_t_annual.csv'),index=False,header=None,sep=',')
Vsb_t_annual.to_csv(str(basin_flag + '_' + time_flag + '_' + routing_method  + '_Vsb_t_annual.csv'),index=False,header=None,sep=',')
Vss_t_annual.to_csv(str(basin_flag + '_' + time_flag + '_' + routing_method  + '_Vss_t_annual.csv'),index=False,header=None,sep=',')
time_mw.to_csv(str(basin_flag + '_' + time_flag + '_' + routing_method  + 'time_mw.csv'),index=False,header=None,sep=',')
tL_mw.to_csv(str(basin_flag + '_' + time_flag + '_' + routing_method  + 'tL_mw.csv'),index=False,header=None,sep=',')
C_ss.to_csv(str(basin_flag + '_' + time_flag + '_' + routing_method  + 'C_ss.csv'),index=False,header=None,sep=',')


1915-01-02 00:00:00


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in double_scalars


1915-01-03 00:00:00


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


1915-01-04 00:00:00


...

1929-08-09 00:00:00
1929-08-10 00:00:00
1929-08-11 00:00:00
1929-08-12 00:00:00
1929-08-13 00:00:00
1929-08-14 00:00:00
1929-08-15 00:00:00
1929-08-16 00:00:00
1929-08-17 00:00:00
1929-08-18 00:00:00
1929-08-19 00:00:00
1929-08-20 00:00:00
1929-08-21 00:00:00
1929-08-22 00:00:00
1929-08-23 00:00:00
1929-08-24 00:00:00
1929-08-25 00:00:00
1929-08-26 00:00:00
1929-08-27 00:00:00
1929-08-28 00:00:00
1929-08-29 00:00:00
1929-08-30 00:00:00
1929-08-31 00:00:00
1929-09-01 00:00:00
1929-09-02 00:00:00
1929-09-03 00:00:00
1929-09-04 00:00:00
1929-09-05 00:00:00
1929-09-06 00:00:00
1929-09-07 00:00:00
1929-09-08 00:00:00
1929-09-09 00:00:00
1929-09-10 00:00:00
1929-09-11 00:00:00
1929-09-12 00:00:00
1929-09-13 00:00:00
1929-09-14 00:00:00
1929-09-15 00:00:00
1929-09-16 00:00:00
1929-09-17 00:00:00
1929-09-18 00:00:00
1929-09-19 00:00:00
1929-09-20 00:00:00
1929-09-21 00:00:00
1929-09-22 00:00:00
1929-09-23 00:00:00
1929-09-24 00:00:00
1929-09-25 00:00:00
1929-09-26 00:00:00
1929-09-27 00:00:00


1930-09-23 00:00:00
1930-09-24 00:00:00
1930-09-25 00:00:00
1930-09-26 00:00:00
1930-09-27 00:00:00
1930-09-28 00:00:00
1930-09-29 00:00:00
1930-09-30 00:00:00
1930-10-01 00:00:00
1930-10-02 00:00:00
1930-10-03 00:00:00
1930-10-04 00:00:00
1930-10-05 00:00:00
1930-10-06 00:00:00
1930-10-07 00:00:00
1930-10-08 00:00:00
1930-10-09 00:00:00
1930-10-10 00:00:00
1930-10-11 00:00:00
1930-10-12 00:00:00
1930-10-13 00:00:00
1930-10-14 00:00:00
1930-10-15 00:00:00
1930-10-16 00:00:00
1930-10-17 00:00:00
1930-10-18 00:00:00
1930-10-19 00:00:00
1930-10-20 00:00:00
1930-10-21 00:00:00
1930-10-22 00:00:00
1930-10-23 00:00:00
1930-10-24 00:00:00
1930-10-25 00:00:00
1930-10-26 00:00:00
1930-10-27 00:00:00
1930-10-28 00:00:00
1930-10-29 00:00:00
1930-10-30 00:00:00
1930-10-31 00:00:00
1930-11-01 00:00:00
1930-11-02 00:00:00
1930-11-03 00:00:00
1930-11-04 00:00:00
1930-11-05 00:00:00
1930-11-06 00:00:00
1930-11-07 00:00:00
1930-11-08 00:00:00
1930-11-09 00:00:00
1930-11-10 00:00:00
1930-11-11 00:00:00


1931-11-07 00:00:00
1931-11-08 00:00:00
1931-11-09 00:00:00
1931-11-10 00:00:00
1931-11-11 00:00:00
1931-11-12 00:00:00
1931-11-13 00:00:00
1931-11-14 00:00:00
1931-11-15 00:00:00
1931-11-16 00:00:00
1931-11-17 00:00:00
1931-11-18 00:00:00
1931-11-19 00:00:00
1931-11-20 00:00:00
1931-11-21 00:00:00
1931-11-22 00:00:00
1931-11-23 00:00:00
1931-11-24 00:00:00
1931-11-25 00:00:00
1931-11-26 00:00:00
1931-11-27 00:00:00
1931-11-28 00:00:00
1931-11-29 00:00:00
1931-11-30 00:00:00
1931-12-01 00:00:00
1931-12-02 00:00:00
1931-12-03 00:00:00
1931-12-04 00:00:00
1931-12-05 00:00:00
1931-12-06 00:00:00
1931-12-07 00:00:00
1931-12-08 00:00:00
1931-12-09 00:00:00
1931-12-10 00:00:00
1931-12-11 00:00:00
1931-12-12 00:00:00
1931-12-13 00:00:00
1931-12-14 00:00:00
1931-12-15 00:00:00
1931-12-16 00:00:00
1931-12-17 00:00:00
1931-12-18 00:00:00
1931-12-19 00:00:00
1931-12-20 00:00:00
1931-12-21 00:00:00
1931-12-22 00:00:00
1931-12-23 00:00:00
1931-12-24 00:00:00
1931-12-25 00:00:00
1931-12-26 00:00:00


1932-12-21 00:00:00
1932-12-22 00:00:00
1932-12-23 00:00:00
1932-12-24 00:00:00
1932-12-25 00:00:00
1932-12-26 00:00:00
1932-12-27 00:00:00
1932-12-28 00:00:00
1932-12-29 00:00:00
1932-12-30 00:00:00
1932-12-31 00:00:00
1933-01-01 00:00:00
1933-01-02 00:00:00
1933-01-03 00:00:00
1933-01-04 00:00:00
1933-01-05 00:00:00
1933-01-06 00:00:00
1933-01-07 00:00:00
1933-01-08 00:00:00
1933-01-09 00:00:00
1933-01-10 00:00:00
1933-01-11 00:00:00
1933-01-12 00:00:00
1933-01-13 00:00:00
1933-01-14 00:00:00
1933-01-15 00:00:00
1933-01-16 00:00:00
1933-01-17 00:00:00
1933-01-18 00:00:00
1933-01-19 00:00:00
1933-01-20 00:00:00
1933-01-21 00:00:00
1933-01-22 00:00:00
1933-01-23 00:00:00
1933-01-24 00:00:00
1933-01-25 00:00:00
1933-01-26 00:00:00
1933-01-27 00:00:00
1933-01-28 00:00:00
1933-01-29 00:00:00
1933-01-30 00:00:00
1933-01-31 00:00:00
1933-02-01 00:00:00
1933-02-02 00:00:00
1933-02-03 00:00:00
1933-02-04 00:00:00
1933-02-05 00:00:00
1933-02-06 00:00:00
1933-02-07 00:00:00
1933-02-08 00:00:00


1934-02-04 00:00:00
1934-02-05 00:00:00
1934-02-06 00:00:00
1934-02-07 00:00:00
1934-02-08 00:00:00
1934-02-09 00:00:00
1934-02-10 00:00:00
1934-02-11 00:00:00
1934-02-12 00:00:00
1934-02-13 00:00:00
1934-02-14 00:00:00
1934-02-15 00:00:00
1934-02-16 00:00:00
1934-02-17 00:00:00
1934-02-18 00:00:00
1934-02-19 00:00:00
1934-02-20 00:00:00
1934-02-21 00:00:00
1934-02-22 00:00:00
1934-02-23 00:00:00
1934-02-24 00:00:00
1934-02-25 00:00:00
1934-02-26 00:00:00
1934-02-27 00:00:00
1934-02-28 00:00:00
1934-03-01 00:00:00
1934-03-02 00:00:00
1934-03-03 00:00:00
1934-03-04 00:00:00
1934-03-05 00:00:00
1934-03-06 00:00:00
1934-03-07 00:00:00
1934-03-08 00:00:00
1934-03-09 00:00:00
1934-03-10 00:00:00
1934-03-11 00:00:00
1934-03-12 00:00:00
1934-03-13 00:00:00


# 4.0 Results

### 4.1. Save model results into a text file at the end of each model run 

If you want to explore the hydrographs created in this application in another software environment, you will run the code below to create a single .csv file that contains modeled hydrograps at each location in the watershed. Please run this code for each run you completed separately. The .csv file can be used in Excel, Matlab or any other software. The name of the new file will be printed.  

### 4.2. Analyze results at different locations

<br /> <img src="https://www.hydroshare.org/resource/d5f1f96849554e03aa98bed5aff36cf7/data/contents/Sauk-SuiattleHeadwaters_Map.JPG" style="float:center;width:600px;padding:40px">


Figure 2. Upper Suiattle River glaciated streamflow network locations


Table 1a. Sediment Transport Model Results at Glaciated streams  

| Glacier name        | USGS Code           |  Drainage Area (sq km)  |
| ------------- |:-------------:| -----:|
| Dusty     | Headwater Suiattle River (HUC 171100060201) | 6.8 |
| Chocolate     | Headwater Suiattle River (HUC 171100060201) | 3.8 |
| Cool     | Headwater Suiattle River (HUC 171100060201) | 1.5 |
| Suiattle     | Headwater Suiattle River (HUC 171100060201) | 5.7 |
| Honeycomb    | Headwater Suiattle River (HUC 171100060201) | 2.7 |
| Vista    | Miners Creek Suiattle River (HUC 171100060202)  | 4.7 |
| Ermine    | Miners Creek Suiattle River (HUC 171100060202)  | 3.3 |

Table 1b. Sediment Transport Model Results at Streamflow Locations in the lower elevation network of the Sauk Watershed with recent streamflow observations


| Location        | USGS Code           | Drainage Area (sq mi)  |
| ------------- |:-------------:| -----:|
| Sauk River near Sauk, WA     | 12189500    | 714 |
| Sauk River at Darrington, WA    | 12187500    |   293 |
| White Chuck River at Darrington, WA    | 12186500    |   78 |
| Sauk River above White Chuck River near Darrington, WA | 12186000    |    152 |

Table 1c. Other Sediment Transport Results at Streamflow Locations in the lower elevation network of the Sauk Watershed

| Location        | USGS Code           | Historic Data  |
| ------------- |:-------------:| -----:|
| Sauk River above Clear Creek near Darrington, WA | 12187000    |  1910-1913 |
| North Fork Sauk River near Barlow Pass, WA| 12185000    |   1917-1920 |


In [ ]:
#totaltransport=Vg_t_annual+Vs_t_annual+Vm_t_annual
#bedloadtransport=Vg_t_annual+Vsb_t_annual

#totaltransport.to_csv(str(basin_flag + '_' + time_flag + '_' + routing_method  + '_totaltransport.csv'),index=False,header=None,sep=',')
#bedloadtransport.to_csv(str(basin_flag + '_' + time_flag + '_' + routing_method  + '_bedloadtransport.csv'),index=False,header=None,sep=',')


In [ ]:
!pwd

In [ ]:
streams_to_plot=[317,329,349,363,368,371]
stream_name=['DustyGlacier','ChocolateGlacier','CoolGlacier','12185000','SuiattleGlacier','HoneycombGlacier']
evaluation_years_plot=np.arange(1915,2011) #np.unique(streamflow.index.year)
i=0
for stream_index in streams_to_plot:  
    i=i+1
    plot_title=''
    fig, (ax1, ax2, ax3, ax4)=plt.subplots(4,1, sharex=True, sharey=False)        
    
    ax1.plot(evaluation_years_plot,
             Vg_mw_annual.loc[stream_index,evaluation_years_plot]/(network.loc[stream_index,'width']*network.loc[stream_index,'segment_length_m']),
             'b-', linewidth=3, label='gravel')
    ax1.plot(evaluation_years_plot,
             Vs_mw_annual.loc[stream_index,evaluation_years_plot]/(network.loc[stream_index,'width']*network.loc[stream_index,'segment_length_m']),
             'r--', linewidth=3, label='sand')
    ax1.set_title('Stream Number '+str(stream_index)+' '+ str(stream_name[i])+'\nSediment Deposited from Landslide',fontsize=14)
    ax1.set_ylabel('Depth (m)',fontsize=12)
    ax1.legend()
    
    ax2.plot(evaluation_years_plot, Vg_cap_annual.loc[stream_index,evaluation_years_plot],'b-', linewidth=3)  
    ax2.plot(evaluation_years_plot, Vs_cap_annual.loc[stream_index,evaluation_years_plot],'r--', linewidth=3)  
    ax2.set_title('Bedload Transport Capacity of Stream',fontsize=14)
    ax2.set_ylabel('Depth (m)',fontsize=12)
    #ax2.set_ylabel('Sediment\nDischarge\n(m3/s)',fontsize=12)
    
    ax3.plot(evaluation_years_plot,Vg_t_annual.loc[stream_index,evaluation_years_plot], 'b-',linewidth=3)
    ax3.plot(evaluation_years_plot,Vs_t_annual.loc[stream_index,evaluation_years_plot], 'r--',linewidth=3)
    ax3.set_title('Sediment Transported Out of Channel',fontsize=14)
    ax3.set_ylabel('Depth (m)',fontsize=12)
    #ax3.set_ylabel('Sediment\nDischarge\n(m3/s)',fontsize=12)
    
    ax4.plot(evaluation_years_plot,
             Vg_b_annual.loc[stream_index,evaluation_years_plot]/(network.loc[stream_index,'width']*network.loc[stream_index,'segment_length_m']),
             'b-', linewidth=3)
    ax4.plot(evaluation_years_plot,
             Vs_b_annual.loc[stream_index,evaluation_years_plot]/(network.loc[stream_index,'width']*network.loc[stream_index,'segment_length_m']),
             'r--', linewidth=3)
    ax4.set_title('Sediment Accumulated on Channel Bed',fontsize=14)
    ax4.set_ylabel('Depth (m)',fontsize=12)
    ax4.set_xlabel('Year',fontsize=16)
    
    sum_transported_gravel=np.sum(Vg_t_annual.loc[stream_index,evaluation_years_plot])
    sum_transported_sand=np.sum(Vs_t_annual.loc[stream_index,evaluation_years_plot])
    sum_transported_mud=np.sum(Vm_t_annual.loc[stream_index,evaluation_years_plot])
    print('Stream Number', stream_index, ', Total gravel transported (10^6 m3)=',sum_transported_gravel/10**6)
    print('Stream Number', stream_index, ', Total sand transported(10^6 m3)=',sum_transported_sand/10**6)
    print('Stream Number', stream_index, ', Total mud transported(10^6 m3)=',sum_transported_mud/10**6)
    print('Stream Number', stream_index, ', Total sediment transported (10^6 m3)=', (sum_transported_gravel+sum_transported_mud+sum_transported_sand)/10**6)
    if stream_index==ref_stream:
        sum_dep_gravel=np.sum(Vg_b_annual.loc[stream_index,evaluation_years_plot[-1]])
        sum_dep_sand=np.sum(Vs_b_annual.loc[stream_index,evaluation_years_plot[-1]])           
        print('Stream Number', stream_index, ', Deposit Sedimentation (10^6 m3)=', (sum_dep_gravel+sum_dep_sand+sum_transported_gravel+sum_transported_sand)*sg/(bd_coarse*10**6))
        print('Stream Number', stream_index, ', Deposit Percent Gravel=', (sum_dep_gravel+sum_transported_gravel)/(sum_dep_gravel+sum_dep_sand+sum_transported_gravel+sum_transported_sand))
        print('Stream Number', stream_index, ', Deposit Percent Sand=', (sum_dep_sand+sum_transported_sand)/(sum_dep_gravel+sum_dep_sand+sum_transported_gravel+sum_transported_sand))
    print('--------------------')


The information, data, or work presented herein was funded in part by the Office of Energy Efficiency and Renewable Energy (EERE), U.S. Department of Energy, under Award Number DE-EE0006506 and the Hydro Research Foundation. Neither the United States Government nor any agency thereof, nor any of their employees, makes and warranty, express or implied, or assumes and legal liability or responsibility for the accuracy, completeness, or usefulness of any information, apparatus, product, or process disclosed, or represents that its use would not infringe privately owned rights. Reference herein to any specific commercial product, process, or service by trade name, trademark, manufacturer, or otherwise does not necessarily constitute or imply its endorsement, recommendation or favoring by the United States Government or any agency thereof. The views and opinions of authors expressed herein do not necessarily state or reflect those of the United States Government or any agency thereof.

[Disclaimer like this needed for BIA funding]

[Christina TO DO - add save back to HydroShare end bits]